In [1]:
!pip install baostock
!pip install pandas
!pip install ta

import baostock as bs
import pandas as pd
import ta
import time

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=33fb8e5b64f49e35d98cc391e441fd52e5f6bbae14fa8316e529b3c464a027e5
  Stored in directory: /Users/huaijinsun/Library/Caches/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [17]:
bs.login()

login success!


In [18]:
def get_stock_data(stock_code, start_date, end_date):

    rs = bs.query_history_k_data_plus(
        stock_code,
        "code,date,open,high,low,close",
        start_date = start_date,
        end_date = end_date,
        frequency = "d",
        adjustflag = "1"
    )
    df = rs.get_data()
    return df

print(get_stock_data("sh.600000", "2025-07-20", "2025-7-24"))

        code        date            open            high             low  \
0  sh.600000  2025-07-21  171.8033188600  172.8244381400  170.5269197600   
1  sh.600000  2025-07-22  171.8033188600  172.4415184100  168.4846812000   
2  sh.600000  2025-07-23  168.4846812000  172.1862385900  168.4846812000   
3  sh.600000  2025-07-24  170.6545596700  170.6545596700  166.5700825500   

            close  
0  172.0585986800  
1  168.8676009300  
2  170.1440000300  
3  166.6977224600  


In [5]:
def mark_MACD_bull(df):
  '''
  MACD指标
  '''
  # 计算MACD线（DIF）
  df["MACD_dif"] = ta.trend.macd(df["close"])

  # 计算MACD信号线（DEA）
  df["MACD_dea"] = ta.trend.macd_signal(df["close"])

  # 计算MACD柱状图（差值）
  df["MACD_hist"] = ta.trend.macd_diff(df["close"])

  # 产生买入信号：当DIF上穿DEA（今天DIF>DEA，昨天DIF<=DEA）
  df["MACD_signal"] = (df["MACD_dif"] > df["MACD_dea"]) & (df["MACD_dif"].shift(1) <= df["MACD_dea"].shift(1))

  golden_cross = (df["MACD_dif"] > df["MACD_dea"]) & (df["MACD_dif"].shift(1) <= df["MACD_dea"].shift(1))
  death_cross = (df["MACD_dif"] < df["MACD_dea"]) & (df["MACD_dif"].shift(1) >= df["MACD_dea"].shift(1))

  df['macd_bull'] = 0
  last_state = 0
  for i in range(len(df)):
      if i == 0:
          df.iloc[i, df.columns.get_loc('macd_bull')] = 0
          continue
      # 只用昨天的状态和信号判断今天的多空
      if golden_cross.iloc[i-1]:
          last_state = 1
      elif death_cross.iloc[i-1]:
          last_state = 0
      df.iloc[i, df.columns.get_loc('macd_bull')] = last_state
  return df

In [6]:
def mark_boll_bull(df):
    window = 20  # Common window for Bollinger Bands
    window_dev = 2 # Common standard deviation multiplier

    # Initialize the BollingerBands indicator
    indicator_bb = ta.volatility.BollingerBands(close=df['close'], window=window, window_dev=window_dev, fillna=False)
    df['upper'] = indicator_bb.bollinger_hband() # High band
    df['lower'] = indicator_bb.bollinger_lband() # Low band
    # 首次突破信号：今天close > upper，昨天close <= upper
    cross_up = (df["close"] > df["upper"]) & (df["close"].shift(1) <= df["upper"].shift(1))
    # 回撤信号：今天close < upper，昨天close >= upper
    cross_down = (df["close"] < df["upper"]) & (df["close"].shift(1) >= df["upper"].shift(1))

    df['boll_bull'] = 0
    state = 0  # 0表示非买入区间，1表示买入区间

    for i in range(len(df)):
        if cross_up.iloc[i]:
            state = 1  # 进入买入区间
        elif cross_down.iloc[i]:
            state = 0  # 离开买入区间
        df.at[df.index[i], 'boll_bull'] = state

    return df

In [7]:
def pass_TD(df):
    '''
    神奇九转（TD Sequential）买入计数逻辑：
    如果当前收盘价 < 4天前的收盘价，计数 +1；
    连续满足9天，触发买入信号。
    '''
    df['TD_count'] = 0
    df['turn9_signal'] = 0  # 初始化信号列

    for i in range(4, len(df)):
        count = 0
        for j in range(9):
            if i - j - 4 < 0:
                break  # 防止越界
            if df["close"].iloc[i - j] < df["close"].iloc[i - j - 4]:
                count += 1
            else:
                break  # 一旦不连续就终止

        df.at[df.index[i], "TD_count"] = count

        if count == 9:
            df.at[df.index[i], "turn9_signal"] = 1

    return df

In [20]:
def get_stock_bull(stock_code, start_date, end_date):

  df = get_stock_data(stock_code, start_date, end_date)
  df["close"] = pd.to_numeric(df["close"], errors='coerce')

  df = mark_MACD_bull(df)
  df = mark_boll_bull(df)
  df = pass_TD(df)
  print(df)

  # Calculate final bull signal
  if not df.empty:
      last_row = df.iloc[-1]
      if last_row['macd_bull'] == 1 and last_row['boll_bull'] == 1 and last_row.get('turn9_signal', 0) == 1:
          return 1
      else:
          return 0
  else:
      return 0

In [21]:
print(get_stock_bull("sh.600001", "2025-06-20", "2025-7-24"))

KeyError: 'close'

In [14]:
data = rs.get_data()
df = pd.DataFrame(data, columns=rs.fields)
print(df.columns)
print(df.head())

Index(['code', 'code_name', 'ipoDate', 'outDate', 'type', 'status'], dtype='object')
Empty DataFrame
Columns: [code, code_name, ipoDate, outDate, type, status]
Index: []


In [ ]:
# 查询所有A股
rs = bs.query_stock_basic()
data = rs.get_data()
df = pd.DataFrame(data, columns=rs.fields)

# 查看结果（code：股票代码，ipoDate: 上市时间, type：1代表股票、2代表指数、3代表ETF、等等，status: 0代表退市状态、1代表可交易状态 ）
# print(df.head())

qualified = []

for code in df['code'][6:1000]:
    try:
        time.sleep(0.5)  # 每个请求之间停 0.5 秒
        result = get_stock_bull(code, "2025-06-20", "2025-07-24")
        if result == 1:
            print(code)
            qualified.append(code)
    except Exception as e:
        print(f"{code} failed: {e}")

print(qualified)

sh.000817 failed: 'close'
sh.000850 failed: 'close'
sh.000973 failed: 'close'
sh.000976 failed: 'close'
sh.000983 failed: 'close'
sh.000996 failed: 'close'
sh.000997 failed: 'close'
sh.000999 failed: 'close'
sh.100001 failed: 'close'
sh.100009 failed: 'close'
sh.100016 failed: 'close'
sh.100087 failed: 'close'
sh.100096 failed: 'close'
sh.100117 failed: 'close'
sh.100177 failed: 'close'
sh.100196 failed: 'close'
sh.100220 failed: 'close'
sh.100236 failed: 'close'
sh.100567 failed: 'close'
sh.100726 failed: 'close'
sh.100795 failed: 'close'
sh.110001 failed: 'close'
sh.110002 failed: 'close'
sh.110003 failed: 'close'
sh.110004 failed: 'close'
sh.110005 failed: 'close'
sh.110006 failed: 'close'
sh.110007 failed: 'close'
sh.110008 failed: 'close'
sh.110009 failed: 'close'
sh.110010 failed: 'close'
sh.110011 failed: 'close'
sh.110012 failed: 'close'
sh.110013 failed: 'close'
sh.110015 failed: 'close'
sh.110016 failed: 'close'
sh.110017 failed: 'close'
sh.110018 failed: 'close'
sh.110019 fa

KeyboardInterrupt: 

In [16]:
bs.logout()

logout success!
